In [1]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer, AutoModelWithLMHead
import torch
import re
from tqdm import tqdm

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
# model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium").cuda()
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2").to(device)

/home/bishal/miniconda3/lib/python3.7/site-packages/transformers/modeling_auto.py:822: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [4]:
with open('daily_dialog/test/dialogues_test.txt') as f, open('outputs/gpt2_responses.txt', 'w') as of:
    lines = f.readlines()
    for line in tqdm(lines):
        dialog = re.split(" *__eou__ *", line.strip())
        
        context = []
        for step, u in enumerate(dialog):
            if len(u) < 1:
                continue
            # print("\nuser:", u)
            u = u + " "
            context.append(u)
            # encode the new user input, add the eos_token and return a tensor in Pytorch
            # new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt').to(device)
            new_user_input_ids = tokenizer.encode(u, return_tensors='pt').to(device)

            # append the new user input tokens to the chat history
            # bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids
            bot_input_ids = torch.cat([bot_input_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

            # generated a response while limiting the total chat history to 1000 tokens, 
            chat_history_ids = model.generate(bot_input_ids, max_length=bot_input_ids.size(1) + 30, pad_token_id=tokenizer.eos_token_id)

            # pretty print last ouput tokens from bot
            resp = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
            
            
            for k, s in enumerate(context):
                of.write(f"Turn {k}: {s}\n")
            of.write(f"GPT2: {resp}\n\n")

  0%|          | 1/1000 [00:56<15:49:02, 57.00s/it]


KeyboardInterrupt: 

In [ ]:
tokenizer.decode(bot_input_ids[0])